## GRAY IMAGES

In [ ]:
from PIL import Image
import pandas as pd
import os

In [ ]:
anot_path = "./data/attention.csv"
images_path = "./data/datasets/"

gray_images_path = "./data/grayimage"

In [ ]:
df = pd.read_csv(anot_path)
df.columns = ['filename', 'score']
df['score'].value_counts()

In [ ]:
save_path = "data/gray_data.csv"

data = {
    'width': [],
    'height': [],
    'file_path': [],
    'score': []
}

if not os.path.exists(gray_images_path):
    os.makedirs(gray_images_path)

for item, score in zip(df['filename'].tolist(), df['score'].tolist()):
    file_path = os.path.join(images_path, item)
    gray_path = os.path.join(gray_images_path, item)

    img = Image.open(file_path).convert("L")
    
    img.save(gray_path)
    
    data['width'].append(img.size[0])
    data['height'].append(img.size[1])
    data['file_path'].append(gray_path)
    data['score'].append(score)

In [ ]:
pd.DataFrame(data).to_csv(save_path, index=False)

## BLUR CANNY CONTOUR 

In [7]:
import pandas as pd
from PIL import Image
import random
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# anomali data 69, 5, 36
# rd_idx = 5

file_path = "data/gray_data.csv"
df = pd.read_csv(file_path)

def Proccess(df = df, idx = None, biar_rapih = False):
    if idx == None:
        idx = random.randint(0, len(df['file_path'].tolist()))
    
    sample_image_name = df['file_path'].tolist()[idx]
    score = df['score'].tolist()[idx]

    img_np = np.array(Image.open(sample_image_name))
    
    if not biar_rapih:
        # gausah didebug ini [biar apa biarin | pusing gw njir ngeliatnya]
        print(f"image idx: {idx} | score: {score} | filename: {sample_image_name}")
        print(f"image size: {img_np.shape}")

    blank = np.zeros(shape=img_np.shape, dtype='uint8')

    gaussBlur = cv.GaussianBlur(img_np, (175, 175), 0.3)
    canny = cv.Canny(gaussBlur, 100, 100)

    _, tresh = cv.threshold(img_np, 125, 255, cv.THRESH_BINARY)
    _, blurtresh = cv.threshold(gaussBlur, 125, 255, cv.THRESH_BINARY)

    contours, _ = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(blank, contours, -1, 255, 1)
    
    images = [img_np, gaussBlur, canny, tresh, blank]
    titles = ['Gray', 'Gaussian Blur', 'Canny', 'Threshold', 'Contours Drawn']
    
    return images, titles

def ShowImage(images, titles):
    
    plt.figure(figsize=(15, 8))

    for i in range(len(images)):
        plt.subplot(2, 3, i + 1)  # 2 baris, 3 kolom
        if len(images[i].shape) == 2:  # grayscale
            plt.imshow(images[i], cmap='gray')
        else:  # RGB image
            plt.imshow(cv.cvtColor(images[i], cv.COLOR_BGR2RGB))
        plt.title(titles[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [8]:
max_w = max(df['width'].tolist())
min_w = min(df['width'].tolist())

max_h = max(df['height'].tolist())
min_h = min(df['height'].tolist())

print(f"max-width: {max_w} | min-width: {min_w} | max-height: {max_h} | min-height: {min_h}")

max-width: 1080 | min-width: 102 | max-height: 550 | min-height: 50


In [9]:
for i in range(0):
    images, titles = Proccess()
    ShowImage(images, titles)

## LOAD DATA INTO NDARRAY

In [10]:
import pandas as pd
import numpy as np
import cv2 as cv

file_path = "data/gray_data.csv"
df = pd.read_csv(file_path)

df.describe()

# mean width = 624
# mean height = 208

# biar 2^ pake width = 512 (2**9)
# biar 2^ pake height = 256 (2**8)

,width,height,score
count,299.000000,299.000000,299.000000
mean,619.033445,204.919732,2.535117
std,217.750917,103.918529,1.400361
min,102.000000,50.000000,0.000000
25%,460.000000,126.000000,1.000000
50%,632.000000,194.000000,3.000000
75%,785.000000,254.500000,4.000000
max,1080.000000,550.000000,4.000000


In [11]:
def resize(np_img, w = 512, h = 256):
    return cv.resize(np_img, dsize=(w, h), interpolation=cv.INTER_LINEAR)

In [12]:
gray_np = [] # gray images
gaussBlur_np = []
canny_np = []
tresh_np = []
contour_np = []

# load to array
for i in range(len(df['file_path'].tolist())):
    '''
    images = [
        img_np, gaussBlur, canny, tresh, blank]
        [gray, gaussblur(gray), canny(gray), tresh(gray), contour(grey)
    ]
    '''
    
    images, _ = Proccess(idx=i, biar_rapih=True)
    # print(type(resize(images[0])))
    # print(resize(images[0]))
    # print(resize(images[0]).shape)
    # break
    gray_np.append(resize(images[0]))
    gaussBlur_np.append(resize(images[1]))
    canny_np.append(resize(images[2]))
    tresh_np.append(resize(images[3]))
    contour_np.append(resize(images[4]))

# transform into 
gray_np = np.array(gray_np)
gaussBlur_np = np.array(gaussBlur_np)
canny_np = np.array(canny_np)
tresh_np = np.array(tresh_np)
contour_np = np.array(contour_np)

In [13]:
print(gray_np.shape, gaussBlur_np.shape, canny_np.shape, tresh_np.shape, contour_np.shape)

# pake dah mau pake yang mana

(299, 256, 512) (299, 256, 512) (299, 256, 512) (299, 256, 512) (299, 256, 512)


## TRY MODEL [alex]

In [14]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

### CNN

In [121]:
import torch
import torch.nn.functional as F
from torch import optim
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [194]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, input_channels=1, num_classes=5):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=8, kernel_size=3, stride=1, padding=1)  # (8, 512, 256)
        self.bn1 = nn.BatchNorm2d(8)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # (8, 256, 128)


        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)  # (16, 256, 128)
        self.bn2 = nn.BatchNorm2d(16)

        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((8,8))
        self.flatten_dim = 1024
        self.fc1 = nn.Linear(self.flatten_dim, 64)
        self.dropout = nn.Dropout(p=0.5)  # Tambahkan dropout
        self.fc2 = nn.Linear(64, num_classes)

    def _initialize_fc1(self, input_channels):
        dummy_input = torch.zeros(1, input_channels, 512, 256)
        x = F.relu(self.conv1(dummy_input))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        flatten_dim = x.view(1,-1).shape[1]
        self.fc1 = nn.Linear(flatten_dim, 64)

    def forward(self, x):
        x = F.relu(self.conv1(x)) #torch.Size([1, 8, 512, 256])
        # print(x.shape)
        x = self.pool(x) #torch.Size([1, 8, 256, 128])
        # print(x.shape)
        x = F.relu(self.conv2(x)) #torch.Size([1, 16, 256, 128])
        # print(x.shape)
        x = self.adaptive_pool(x)#torch.Size([1, 16, 8, 8])
        # print(x.shape)
        x = x.view(x.size(0), -1)  # Flatten #torch.Size([1, 1024])
        # print(x.shape)
        x = F.relu(self.fc1(x))
        # print(x.shape)
        x = self.dropout(x)  # Terapkan dropout hanya saat training
        # print(x.shape)
        x = self.fc2(x)
        # print(x.shape)

        return x

In [195]:
# one-hot encode
y_one_hot_encode = []
for i in df['score'].tolist():
    onehot_temp = [0]*5
    onehot_temp[i] = 1
    y_one_hot_encode.append(onehot_temp)

In [196]:
from sklearn.model_selection import train_test_split
import torch

X = tresh_np.reshape(-1, 1, 512, 256).astype(np.float32)
X = X / 255.0
y = np.array(y_one_hot_encode).astype(np.float32)

# convert data to tensor
X = torch.tensor(X)
y = torch.tensor(y, dtype=torch.float64)

# Split sebelum konversi ke tensor
X_train, X_test, y_train, y_test = train_test_split(X, y_int, test_size=0.2, random_state=42)

# Konversi ke tensor
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Cek shape
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([239, 1, 512, 256]) torch.Size([239])
torch.Size([60, 1, 512, 256]) torch.Size([60])


In [197]:
model = CNN()

sample_input = X_train[0].unsqueeze(0)

model(sample_input)

print(y_train)

tensor([4, 4, 2, 0, 0, 1, 4, 1, 1, 3, 3, 0, 4, 4, 1, 4, 4, 2, 3, 0, 4, 0, 3, 2,
        0, 2, 2, 4, 1, 4, 1, 1, 4, 4, 3, 1, 2, 4, 1, 2, 2, 0, 1, 4, 2, 4, 0, 4,
        4, 1, 3, 2, 0, 4, 2, 1, 3, 3, 4, 1, 3, 3, 1, 0, 1, 1, 1, 3, 2, 4, 2, 4,
        2, 1, 4, 3, 2, 4, 4, 4, 1, 2, 2, 2, 4, 4, 2, 2, 4, 4, 1, 4, 4, 4, 2, 3,
        4, 4, 3, 4, 2, 4, 4, 1, 4, 4, 1, 1, 4, 4, 1, 4, 3, 4, 1, 1, 4, 2, 4, 2,
        2, 4, 1, 0, 4, 2, 1, 3, 0, 1, 4, 2, 2, 1, 4, 2, 1, 3, 3, 1, 4, 3, 4, 2,
        4, 4, 4, 4, 2, 4, 4, 1, 0, 3, 3, 4, 4, 1, 1, 4, 3, 3, 4, 0, 4, 2, 2, 4,
        4, 4, 3, 1, 3, 4, 3, 3, 4, 3, 4, 1, 2, 3, 4, 3, 3, 4, 4, 1, 4, 2, 4, 1,
        1, 4, 2, 0, 2, 3, 4, 4, 2, 4, 1, 3, 4, 0, 4, 4, 2, 3, 1, 4, 4, 1, 1, 2,
        4, 1, 2, 0, 0, 4, 2, 0, 4, 1, 4, 1, 4, 4, 4, 1, 0, 0, 4, 2, 4, 1, 4])


In [198]:
dataset = TensorDataset(X_train, y_train)
trainloader = DataLoader(dataset, batch_size=12, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
testloader = DataLoader(test_dataset, batch_size=12, shuffle=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [199]:
torch.tensor(y[0]).shape

torch.Size([5])

In [200]:
# model = CNN().to(device=device)

In [201]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch

labels = df['score'].values

class_labels = np.unique(labels)

weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels)
class_weights = torch.tensor(weights, dtype=torch.float).to(device)

print(f"Class Weights : {class_weights}")

Class Weights : tensor([2.0621, 1.0873, 1.0873, 1.2723, 0.5292], device='cuda:0')


In [202]:
import torch.optim as optim

# Hyperparameter
epochs = 10   # Coba dengan lebih banyak epoch untuk melihat apakah model dapat lebih baik
learning_rate = 0.001  # Turunkan learning rate untuk memperbaiki konvergensi
batch_size = 64

model = CNN().to(device=device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

criterion = nn.CrossEntropyLoss()

In [203]:
patience = 2
best_loss = float('inf')
counter = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(tqdm(trainloader)):
        data = data.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        pred_res = model(data)
        loss = criterion(pred_res, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    avg_loss = running_loss / len(trainloader)
    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f}")
    
    # ===== VALIDASI =====
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_data, val_target in testloader:
            val_data = val_data.to(device)
            val_target = val_target.to(device)
            val_output = model(val_data)
            v_loss = criterion(val_output, val_target)
            val_loss += v_loss.item()
    val_loss /= len(testloader)
    print(f"Validation Loss: {val_loss:.4f}")
    
    # ===== EARLY STOPPING =====
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), 'best_model.pt')  # simpan model terbaik
        print("Model improved. Saving model.")
    else:
        counter += 1
        print(f"No improvement. EarlyStopping counter: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            break

100%|██████████| 20/20 [00:00<00:00, 33.57it/s]


Epoch 1/10 - Train Loss: 1.5641
Validation Loss: 1.5787
Model improved. Saving model.


100%|██████████| 20/20 [00:00<00:00, 66.71it/s]


Epoch 2/10 - Train Loss: 1.5071
Validation Loss: 1.5856
No improvement. EarlyStopping counter: 1/2


100%|██████████| 20/20 [00:00<00:00, 67.87it/s]


Epoch 3/10 - Train Loss: 1.5083
Validation Loss: 1.5698
Model improved. Saving model.


100%|██████████| 20/20 [00:00<00:00, 67.09it/s]


Epoch 4/10 - Train Loss: 1.5180
Validation Loss: 1.5712
No improvement. EarlyStopping counter: 1/2


100%|██████████| 20/20 [00:00<00:00, 67.26it/s]

Epoch 5/10 - Train Loss: 1.5032
Validation Loss: 1.6069
No improvement. EarlyStopping counter: 2/2
Early stopping triggered.


In [183]:
torch.save(model.state_dict(), 'model/model-again.pth')

In [184]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = TensorDataset(X_train, y_train)
trainloader = DataLoader(dataset, batch_size=32, shuffle=True)

loaded_model = CNN()
loaded_model.load_state_dict(torch.load('model/model1.pth'))
loaded_model.to(device)

loaded_model.eval()

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (adaptive_pool): AdaptiveAvgPool2d(output_size=(8, 8))
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=64, out_features=5, bias=True)
)

In [185]:
rd_idx = random.randint(0, 198)

sample_input = X_train[rd_idx].unsqueeze(0).to(device)
sample_output = y_train[rd_idx]
print(sample_input.shape)

pred_result = loaded_model(sample_input)
print(pred_result, sample_output)

torch.Size([1, 1, 512, 256])
tensor([[ 0.0046,  0.0285, -0.0270, -0.0891,  0.0051]], device='cuda:0',
       grad_fn=<AddmmBackward0>) tensor(0)


In [186]:
def check_acc(loader, model, device):
    num_correct = 0
    num_samples = 0

    model.eval()  # Set model ke mode evaluasi
    with torch.no_grad():  # Tidak perlu track gradient
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)  # forward
            _, preds = scores.max(1)  # ambil index prediksi tertinggi

            num_correct += (preds == y).sum().item()  # kalau target one-hot
            num_samples += y.size(0)

    acc = num_correct / num_samples
    print(f"Accuracy: {acc*100:.2f}%")
    
    model.train()  # Kembalikan ke mode training setelah evaluasi


In [187]:
check_acc(testloader, loaded_model, device)

Accuracy: 8.33%


In [ ]:
check_acc(trainloader, loaded_model, device)

In [ ]:
check_acc(trainloader, loaded_model, device)

In [ ]:
df['score'].value_counts()